In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [3]:
import pandas as pd
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dreaddit-test.csv")
test.describe()

,id,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,lex_liwc_Dic,lex_liwc_function,lex_liwc_pronoun,lex_liwc_ppron,lex_liwc_i,lex_liwc_we,lex_liwc_you,lex_liwc_shehe,lex_liwc_they,lex_liwc_ipron,lex_liwc_article,lex_liwc_prep,lex_liwc_auxverb,lex_liwc_adverb,lex_liwc_conj,lex_liwc_negate,lex_liwc_verb,lex_liwc_adj,lex_liwc_compare,lex_liwc_interrog,lex_liwc_number,lex_liwc_quant,lex_liwc_affect,lex_liwc_posemo,lex_liwc_negemo,lex_liwc_anx,lex_liwc_anger,...,lex_liwc_motion,lex_liwc_space,lex_liwc_time,lex_liwc_work,lex_liwc_leisure,lex_liwc_home,lex_liwc_money,lex_liwc_relig,lex_liwc_death,lex_liwc_informal,lex_liwc_swear,lex_liwc_netspeak,lex_liwc_assent,lex_liwc_nonflu,lex_liwc_filler,lex_liwc_AllPunc,lex_liwc_Period,lex_liwc_Comma,lex_liwc_Colon,lex_liwc_SemiC,lex_liwc_QMark,lex_liwc_Exclam,lex_liwc_Dash,lex_liwc_Quote,lex_liwc_Apostro,lex_liwc_Parenth,lex_liwc_OtherP,lex_dal_max_pleasantness,lex_dal_max_activation,lex_dal_max_imagery,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
count,715.000000,715.000000,715.000000,7.150000e+02,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,...,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.00000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000
mean,13861.372028,0.516084,0.717895,1.517570e+09,24.627972,4.705624,85.667133,34.743399,38.410839,69.917413,32.375706,17.861091,14.885566,92.513706,58.890937,20.054713,14.110042,9.360406,0.780629,0.775259,2.619860,0.573510,5.929762,4.828923,13.498098,10.234196,6.169497,7.735692,2.277483,19.270545,4.529538,2.336797,1.520993,1.375986,2.320895,6.050853,2.606573,3.360769,0.980853,0.880308,...,1.975552,6.328685,6.415217,1.765329,0.768364,0.573035,0.720657,0.133413,0.143301,0.820769,0.251287,0.219580,0.128378,0.112238,0.054895,16.733203,6.198294,3.568420,0.185538,0.099189,0.460196,0.162937,0.468797,0.425399,3.325063,0.737287,1.102014,2.799170,2.713098,2.944056,1.081046,1.117715,1.00028,1.723382,1.529951,1.878992,0.856294,8.951049,5.497574,0.045345
std,17765.755427,0.500091,0.325688,1.553944e+07,113.086473,3.244652,31.197320,26.331580,30.834289,31.682491,34.495223,7.002507,5.154319,4.458381,6.213774,5.547130,4.739893,4.440333,1.592174,1.986996,3.636421,1.348621,3.233104,2.639801,3.644833,3.624511,3.186709,2.914734,1.882514,4.499541,2.703421,1.910232,1.518803,1.921668,1.877378,3.236959,2.364876,2.810139,1.532993,1.332447,...,1.865840,3.349923,3.800014,2.376354,1.414370,1.130960,1.728165,0.616299,0.493143,1.314436,0.692429,0.707232,0.433971,0.381352,0.293046,9.101795,3.147372,2.759322,0.614649,0.482570,1.185406,0.741345,2.091756,1.247314,2.838065,1.538606,6.207591,0.158079,0.172314,0.131370,0.107884,0.089972,0.00748,0.048894,0.100724,0.056044,0.174352,12.891993,2.490971,0.194114
min,2.000000,0.000000,0.000000,1.483330e+09,0.000000,-4.800000,12.000000,1.000000,1.000000,1.000000,1.000000,2.400000,0.000000,66.670000,16.670000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.360000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [4]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dreaddit-train.csv")
train.describe()

,id,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,lex_liwc_WPS,lex_liwc_Sixltr,lex_liwc_Dic,lex_liwc_function,lex_liwc_pronoun,lex_liwc_ppron,lex_liwc_i,lex_liwc_we,lex_liwc_you,lex_liwc_shehe,lex_liwc_they,lex_liwc_ipron,lex_liwc_article,lex_liwc_prep,lex_liwc_auxverb,lex_liwc_adverb,lex_liwc_conj,lex_liwc_negate,lex_liwc_verb,lex_liwc_adj,lex_liwc_compare,lex_liwc_interrog,lex_liwc_number,lex_liwc_quant,lex_liwc_affect,lex_liwc_posemo,lex_liwc_negemo,lex_liwc_anx,lex_liwc_anger,...,lex_liwc_motion,lex_liwc_space,lex_liwc_time,lex_liwc_work,lex_liwc_leisure,lex_liwc_home,lex_liwc_money,lex_liwc_relig,lex_liwc_death,lex_liwc_informal,lex_liwc_swear,lex_liwc_netspeak,lex_liwc_assent,lex_liwc_nonflu,lex_liwc_filler,lex_liwc_AllPunc,lex_liwc_Period,lex_liwc_Comma,lex_liwc_Colon,lex_liwc_SemiC,lex_liwc_QMark,lex_liwc_Exclam,lex_liwc_Dash,lex_liwc_Quote,lex_liwc_Apostro,lex_liwc_Parenth,lex_liwc_OtherP,lex_dal_max_pleasantness,lex_dal_max_activation,lex_dal_max_imagery,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
count,2838.000000,2838.000000,2838.000000,2.838000e+03,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.00000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,...,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000,2838.000000
mean,13751.999295,0.524313,0.808972,1.518107e+09,18.262156,4.684272,85.996124,35.240941,40.948231,67.044249,33.428157,18.189475,14.858157,92.349292,58.598887,19.767699,13.987530,9.066254,0.764679,0.865772,2.708076,0.582667,5.76902,4.937636,13.356656,10.306311,6.045853,7.599056,2.262903,19.431293,4.362734,2.287287,1.608217,1.444151,2.246406,6.097400,2.697861,3.304732,0.914197,0.928534,...,2.004397,6.295144,6.199316,1.875694,0.785134,0.635113,0.786226,0.115662,0.141811,0.825655,0.246483,0.255201,0.119112,0.124767,0.049856,17.053650,6.066293,3.573312,0.218788,0.123428,0.519066,0.192491,0.486801,0.473513,3.176459,0.778961,1.444264,2.796964,2.704940,2.948414,1.088001,1.120099,1.000211,1.722759,1.536400,1.879385,0.843517,9.948555,5.448836,0.040740
std,17340.161897,0.499497,0.177038,1.552209e+07,79.419166,3.316435,32.334887,26.486189,31.587117,32.880644,35.334770,9.516060,5.531303,5.301902,6.852503,5.577772,4.792445,4.653986,1.564468,2.095719,3.611687,1.265140,3.22379,2.611391,3.649962,3.649751,3.182057,2.813245,1.894125,4.769641,2.688372,1.990455,1.540092,1.910695,1.790213,3.530977,2.541304,3.021693,1.468605,1.447357,...,1.796235,3.261277,3.781227,2.455234,1.352407,1.217066,1.716934,0.528442,0.555553,1.319571,0.743807,0.730756,0.484504,0.415866,0.328158,12.335859,3.219358,2.870731,0.807024,0.500141,2.342935,1.029934,1.805352,1.295909,2.723686,1.728138,10.057253,0.162649,0.174462,0.125843,0.117159,0.085227,0.006500,0.047835,0.102971,0.058932,0.174794,21.798032,2.535829,0.195490
min,4.000000,0.000000,0.428571,1.483274e+09,0.000000,-6.620000,5.000000,1.000000,1.000000,1.000000,1.000000,2.400000,0.000000,27.910000,4.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [5]:
train['subreddit'].unique()

array(['ptsd', 'assistance', 'relationships', 'survivorsofabuse',
       'domesticviolence', 'anxiety', 'homeless', 'stress',
       'almosthomeless', 'food_pantry'], dtype=object)

In [6]:
new_train = train[['subreddit','text','label','sentiment']]
new_train = new_train[new_train['subreddit']!='food_pantry']
index = new_train[new_train['subreddit']=='almosthomeless'].index
new_train.loc[index,'subreddit'] = 'homeless'

In [7]:
new_train['subreddit'].unique()

array(['ptsd', 'assistance', 'relationships', 'survivorsofabuse',
       'domesticviolence', 'anxiety', 'homeless', 'stress'], dtype=object)

In [8]:
train_x,train_y = new_train[['text','sentiment','label']], new_train[['subreddit']]

In [9]:
new_test = test[['subreddit','text','label','sentiment']]
new_test = new_test[new_test['subreddit']!='food_pantry']
index = new_test[new_test['subreddit']=='almosthomeless'].index
new_test.loc[index,'subreddit'] = 'homeless'

In [10]:
new_test['subreddit'].unique()

array(['relationships', 'anxiety', 'ptsd', 'assistance', 'homeless',
       'domesticviolence', 'survivorsofabuse', 'stress'], dtype=object)

In [11]:
test_x,test_y = new_test[['text','sentiment','label']],new_test[['subreddit']]

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenize our training data
num_words = 100000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(train_x['text'])

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(train_x['text'])

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# Output the results of our work
print("Word index:\n", word_index)
print("\nTraining sequences:\n", train_sequences)
print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(train_sequences))
print("Padded Training sequences data type:", type(train_padded))

Word index:
 {'<UNK>': 1, 'i': 2, 'to': 3, 'and': 4, 'the': 5, 'a': 6, 'my': 7, 'of': 8, 'me': 9, 'that': 10, 'in': 11, 'it': 12, 'for': 13, 'was': 14, 'is': 15, 'but': 16, 'have': 17, 'with': 18, 'this': 19, 'he': 20, 'on': 21, 'so': 22, 'her': 23, 'she': 24, 'you': 25, 'be': 26, "i'm": 27, 'not': 28, 'just': 29, 'about': 30, 'or': 31, 'like': 32, 'at': 33, 'we': 34, 'if': 35, 'out': 36, 'as': 37, 'do': 38, 'up': 39, 'him': 40, 'what': 41, 'had': 42, 'because': 43, 'when': 44, 'know': 45, 'been': 46, 'get': 47, 'all': 48, 'time': 49, 'are': 50, 'from': 51, 'feel': 52, 'can': 53, 'would': 54, 'am': 55, 'has': 56, "don't": 57, 'they': 58, 'an': 59, 'really': 60, 'how': 61, 'his': 62, 'even': 63, 'now': 64, 'one': 65, 'want': 66, 'will': 67, 'help': 68, 'some': 69, 'no': 70, 'go': 71, "i've": 72, 'more': 73, 'them': 74, 'who': 75, "it's": 76, 'people': 77, 'back': 78, 'there': 79, 'going': 80, 'any': 81, 'being': 82, 'anxiety': 83, 'years': 84, 'over': 85, 'think': 86, 'i’m': 87, 'things

In [14]:
import pickle

# saving
with open('/content/drive/MyDrive/Colab Notebooks/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # loading
# with open('tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

In [16]:
test_sequences = tokenizer.texts_to_sequences(test_x['text'])
test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

In [17]:
vocab_size=13000

In [18]:
train_y['subreddit'] = train_y.subreddit.astype('category')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_y["subreddit"] = le.fit_transform(train_y["subreddit"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
test_y['subreddit'] = test_y.subreddit.astype('category')
le = LabelEncoder()
test_y["subreddit"] = le.fit_transform(test_y["subreddit"])

In [21]:
from keras.utils import to_categorical
test_labels = to_categorical(test_y['subreddit'])
train_labels = to_categorical(train_y['subreddit'])

In [25]:
embed_dim = 32  # Embedding size for each token
num_heads = 8  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
history = model.fit(train_padded,train_x['label'], batch_size=5, epochs=25, validation_data=(test_padded, test_x['label']))

In [29]:
reconstructed_model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/StressDetector')

In [28]:
model.save("/content/drive/MyDrive/Colab Notebooks/StressDetector")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/StressDetector/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/StressDetector/assets


In [34]:
def stress_detect(msg):
  test_sequences = tokenizer.texts_to_sequences([msg])
  test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
  ans = model.predict(test_padded)
  return ans[0][0]

0.44148374